# <center> <img src="../../img/ITESOLogo.png" alt="ITESO" width="480" height="130"> </center>
# <center> **Departamento de Electrónica, Sistemas e Informática** </center>
---
## <center> Computer Systems Engineering  </center>
---
### <center> Big Data Processing </center>
---
#### <center> **Autumn 2025** </center>
---

**Lab 04**: Data Unions & Joins Pipeline

**Date**: September 23rd 2025

**Student Name**: Francisco

**Professor**: Pablo Camarillo Ramirez

In [1]:
# Create SparkSession

In [3]:
import findspark
findspark.init()

from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("Examples on data joins") \
    .master("spark://9eae4bedccb3:7077") \
    .config("spark.ui.port", "4040") \
    .getOrCreate()

sc = spark.sparkContext
sc.setLogLevel("ERROR")

In [4]:
from FrankModule.spark_utils import SparkUtils
schema_ag = SparkUtils.generate_schema([("id_ag", "int"), ("agency_info", "string")])
df_agency = spark.read.schema(schema_ag).option("header", True).csv("/opt/spark/work-dir/data/car_service/agencies")
df_agency.show(truncate=False)


[Stage 0:>                                                          (0 + 1) / 1]

+-----+-----------------------------------------------------+
|id_ag|agency_info                                          |
+-----+-----------------------------------------------------+
|1    |{'agency_name': 'NYC Rentals', 'city': 'New York'}   |
|2    |{'agency_name': 'LA Car Rental', 'city': 'Londres'}  |
|3    |{'agency_name': 'Zapopan Auto', 'city': 'Zapopan'}   |
|4    |{'agency_name': 'SF Cars', 'city': 'San Francisco'}  |
|5    |{'agency_name': 'Mexico Cars', 'city': 'Mexico City'}|
+-----+-----------------------------------------------------+



In [5]:
schema_ag = SparkUtils.generate_schema([("rental_id", "int"), ("rental_info", "string")])
df_rental = spark.read.schema(schema_ag).option("header", True).csv("/opt/spark/work-dir/data/car_service/rentals")
df_rental.show(truncate=False)

+---------+--------------------------------------------------+
|rental_id|rental_info                                       |
+---------+--------------------------------------------------+
|11891    |{'car_id': 21, 'customer_id': 71, 'agency_id': 1} |
|11892    |{'car_id': 11, 'customer_id': 52, 'agency_id': 2} |
|11893    |{'car_id': 22, 'customer_id': 116, 'agency_id': 4}|
|11894    |{'car_id': 5, 'customer_id': 107, 'agency_id': 1} |
|11895    |{'car_id': 4, 'customer_id': 53, 'agency_id': 4}  |
|11896    |{'car_id': 8, 'customer_id': 131, 'agency_id': 2} |
|11897    |{'car_id': 23, 'customer_id': 66, 'agency_id': 3} |
|11898    |{'car_id': 24, 'customer_id': 60, 'agency_id': 4} |
|11899    |{'car_id': 27, 'customer_id': 92, 'agency_id': 2} |
|11900    |{'car_id': 3, 'customer_id': 40, 'agency_id': 4}  |
|11901    |{'car_id': 3, 'customer_id': 35, 'agency_id': 1}  |
|11902    |{'car_id': 3, 'customer_id': 72, 'agency_id': 1}  |
|11903    |{'car_id': 29, 'customer_id': 58, 'agency_id

In [6]:
schema_ag = SparkUtils.generate_schema([("custumer_id", "int"), ("custumer_info", "string")])
df_customer = spark.read.schema(schema_ag).option("header", True).csv("/opt/spark/work-dir/data/car_service/customers")
df_customer.show(truncate=False)

+-----------+-------------------------------------------------------------------------+
|custumer_id|custumer_info                                                            |
+-----------+-------------------------------------------------------------------------+
|1          |{'customer_name': 'Tiffany Riley', 'city': 'Monterrey', 'age': 32}       |
|2          |{'customer_name': 'Matthew Davies', 'city': 'Monterrey', 'age': 36}      |
|3          |{'customer_name': 'Rebecca Miller', 'city': 'Mexico City', 'age': 30}    |
|4          |{'customer_name': 'Katelyn Mccoy', 'city': 'New York', 'age': 34}        |
|5          |{'customer_name': 'Dana Dennis', 'city': 'Zapopan', 'age': 26}           |
|6          |{'customer_name': 'Daniel Norton', 'city': 'Mexico City', 'age': 34}     |
|7          |{'customer_name': 'Robert Garcia', 'city': 'Zapopan', 'age': 47}         |
|8          |{'customer_name': 'Michael Williams', 'city': 'Monterrey', 'age': 33}    |
|9          |{'customer_name': '

In [7]:
schema_ag = SparkUtils.generate_schema([("car_id", "int"), ("car_info", "string")])
df_car = spark.read.schema(schema_ag).option("header", True).csv("/opt/spark/work-dir/data/car_service/cars")
df_car.show(truncate=False)

+------+----------------------------------------------------------------------------------------+
|car_id|car_info                                                                                |
+------+----------------------------------------------------------------------------------------+
|1     |{'car_name': 'Chang-Fisher Model 7', 'brand_id': 5, 'price_per_day': 139}               |
|2     |{'car_name': 'Sheppard-Tucker Model 4', 'brand_id': 6, 'price_per_day': 70}             |
|3     |{'car_name': 'Faulkner-Howard Model 5', 'brand_id': 3, 'price_per_day': 53}             |
|4     |{'car_name': 'Wagner LLC Model 1', 'brand_id': 5, 'price_per_day': 89}                  |
|5     |{'car_name': 'Campos PLC Model 4', 'brand_id': 4, 'price_per_day': 112}                 |
|6     |{'car_name': 'Archer-Patel Model 2', 'brand_id': 3, 'price_per_day': 55}                |
|7     |{'car_name': 'Patrick, Barrera and Collins Model 6', 'brand_id': 6, 'price_per_day': 66}|
|8     |{'car_name':

In [14]:
from pyspark.sql.functions import get_json_object, col

# Agencies
df_agencies = df_agency \
    .withColumn("agency_name", get_json_object(col("agency_info"), "$.agency_name"))

# Cars
df_cars = df_car \
    .withColumn("car_name", get_json_object(col("car_info"), "$.car_name"))

# Customers
df_customers = df_customer \
    .withColumn("customer_name", get_json_object(col("custumer_info"), "$.customer_name"))

# Rentals
df_rentals = df_rental \
    .withColumn("car_id", get_json_object(col("rental_info"), "$.car_id")) \
    .withColumn("customer_id", get_json_object(col("rental_info"), "$.customer_id")) \
    .withColumn("agency_id", get_json_object(col("rental_info"), "$.agency_id"))

df_agencies.show()
df_cars.show()
df_customers.show()
df_rentals.show()

+-----+--------------------+-------------+
|id_ag|         agency_info|  agency_name|
+-----+--------------------+-------------+
|    1|{'agency_name': '...|  NYC Rentals|
|    2|{'agency_name': '...|LA Car Rental|
|    3|{'agency_name': '...| Zapopan Auto|
|    4|{'agency_name': '...|      SF Cars|
|    5|{'agency_name': '...|  Mexico Cars|
+-----+--------------------+-------------+

+------+--------------------+--------------------+
|car_id|            car_info|            car_name|
+------+--------------------+--------------------+
|     1|{'car_name': 'Cha...|Chang-Fisher Model 7|
|     2|{'car_name': 'She...|Sheppard-Tucker M...|
|     3|{'car_name': 'Fau...|Faulkner-Howard M...|
|     4|{'car_name': 'Wag...|  Wagner LLC Model 1|
|     5|{'car_name': 'Cam...|  Campos PLC Model 4|
|     6|{'car_name': 'Arc...|Archer-Patel Model 2|
|     7|{'car_name': 'Pat...|Patrick, Barrera ...|
|     8|{'car_name': 'Jon...|Jones, Jefferson ...|
|     9|{'car_name': 'Gar...|Garcia, Hamilton ...|


In [20]:
from pyspark.sql.functions import col

# Join Rentals con Cars
df_rentals = df_rentals.join(df_cars, on="car_id", how="left")

# Join con Agencies 
df_rentals = df_rentals.join(df_agencies,on="id_ag", how="left")

# Join con Customers
df_rentals = df_rentals.join(df_customers,on="customers_id", how="left")

df_rentals.show()


AnalysisException: [UNRESOLVED_USING_COLUMN_FOR_JOIN] USING column `car_id` cannot be resolved on the left side of the join. The left-side columns: [`agency_id_fk`, `car_id_fk`, `customer_id_fk`, `rental_id`, `rental_info`]. SQLSTATE: 42703

In [21]:
final_df = df_join.select(
    col("rental_id"),
    col("car_name"),
    col("agency_name"),
    col("customer_name")
)

NameError: name 'df_join' is not defined

In [22]:
sc.stop()